---
layout: post
title: Tri 1 Individual Review
toc: true
comments: true
description: A file for the individual review for Tri 1
courses: { csa: {week: 9} }
type: tangibles
---

## Individual Review

- [ ] Plans to study for college board quiz 
   - [ ] FRQ's best way to study
   - [ ] Trying difrent types of FRQ's and using them to understand and learn for quiz
   - [ ] [FRQ blog](https://firestorm0986.github.io/CSA-blog-site//2023/09/18/sample-frq1_IPYNB_2_.html)

---

- [ ] [Schedule/Timebox](https://firestorm0986.github.io/CSA-blog-site/csa)
- [ ] project
   - [ ] [Setting up signin and login ](https://github.com/aidenhuynh/cj_frontend/issues/4) : OOP Java
       - [ ] Cutting down on the person files
       - [ ] modleinit.java file
       - [ ] class code generation post and modleinit.java
   - [ ] [setting up JWT](https://github.com/aidenhuynh/cj_backend/issues/5) : OOP java
       - [ ] Creating HumandetailService and redoing the functions to match. like findbyusername()
       - [ ] Adding to the JWT files --> needed to understand each code segement and navigate through various files and function definitions to find answer
       - [ ] various circular dependencies decleration like in securityconfig due to person and human api conflicts
   - [ ] [Deployment](https://github.com/aidenhuynh/cj_frontend/issues/8) : AWS
       - [ ] Using newer version --> took research

---

- [ ] Github analytics
   - [ ] [site](https://github.com/Firestorm0986)
   - [ ] [One of the better commits](https://github.com/aidenhuynh/cj_backend/commit/5a6dcc598d78cdf4e6fe9337431ec012ac1a9099)

### Project Commits: 
![image](https://github.com/Firestorm0986/CSA-blog-site/assets/108041389/05f7b5c2-ca24-4076-b178-525cf2df59ee)
![image](https://github.com/Firestorm0986/CSA-blog-site/assets/108041389/a090b76a-edb9-4e89-ab13-9b78c0e8a9fd)

### Github Analytics:
![image](https://github.com/Firestorm0986/CSA-blog-site/assets/108041389/cb729de4-724a-4d26-89c4-b86f9ea2c74c)

### CSA Blog site commits:
![image](https://github.com/Firestorm0986/CSA-blog-site/assets/108041389/f9f43196-311a-4ec6-85c7-c6474cfdf105)



## Detailed Overview of Human API
The main sign-in page of out team project uses the ``Human api``. 

# The API

- [x] Set up JWT on human api 
   - [x] Create humanDetailsService.java file and more 1 1/2 hour
   - [x] Update JWT files and security file- 1 1/2 hr
- [x] test JWT and see if token is sent - 30 mins (forgot it sends in headers 😅 )
- [x] Set roles and specific authorities (maybe)
- [x] test new JWT and role

## General idea:


![Image](https://github.com/aidenhuynh/cj_backend/assets/108041389/e7b9436e-809f-4c0f-b41f-ebe5d2f402a0)

## Details:


![Image](https://github.com/aidenhuynh/cj_backend/assets/108041389/ea633fdc-8afc-4e58-858a-4c0f451cc272)







### Creating the API

I strtedd with simple columns similar to those in Person and slowly changed them to match my needs. The main ones changed are `role` and `classcodes`. once this was done I created the APIcontroller file and JPArepository file. Below is the **Human.java** file.

In [ ]:

@Data
@AllArgsConstructor
@NoArgsConstructor
@Entity
@TypeDef(name="json", typeClass = JsonType.class)
public class Human {

    // automatic unique identifier for Person record
    @Id
    @GeneratedValue(strategy = GenerationType.AUTO)
    private Long id;

    // email, password, roles are key attributes to login and authentication
    @NotEmpty
    @Size(min=5)
    @Column(unique=true)
    @Email
    private String email;

    @NotEmpty
    private String password;

    // @NonNull, etc placed in params of constructor: "@NonNull @Size(min = 2, max = 30, message = "Name (2 to 30 chars)") String name"
    @NonNull
    @Size(min = 2, max = 30, message = "Name (2 to 30 chars)")
    private String name;

    @DateTimeFormat(pattern = "yyyy-MM-dd")
    private Date dob;

    private String classCode;

    @NonNull
    private String role;
    /* HashMap is used to store JSON for daily "stats"
    "stats": {
        "2022-11-13": {
            "calories": 2200,
            "steps": 8000
        }
    }
    */
    

    // Constructor used when building object from an API
    public Human(String email, String password, String name, Date dob, String role) {
        this.email = email;
        this.password = password;
        this.name = name;
        this.dob = dob;
        this.role = role;
        
    }

    // A custom getter to return age from dob attribute
    public int getAge() {
        if (this.dob != null) {
            LocalDate birthDay = this.dob.toInstant().atZone(ZoneId.systemDefault()).toLocalDate();
            return Period.between(birthDay, LocalDate.now()).getYears(); }
        return -1;
    }

    // Initialize static test data 
    public static Human[] init() {

        // basics of class construction
        Human p1 = new Human();
        p1.setName("Thomas Edison");
        p1.setEmail("toby@gmail.com");
        p1.setPassword("123Toby!");
        p1.setRole("Teacher");
        // adding Note to notes collection
        try {  // All data that converts formats could fail
            Date d = new SimpleDateFormat("MM-dd-yyyy").parse("01-01-1840");
            p1.setDob(d);
        } catch (Exception e) {
            // no actions as dob default is good enough
        }

    public static void main(String[] args) {
        // obtain Person from initializer
        Human persons[] = init();

        // iterate using "enhanced for loop"
        for( Human person : persons) {
            System.out.println(person);  // print object
        }
    }

}

### HumanApiController 
The API controller file for the Human class or api had a major change similar to modleinit.java. I had to auto generate a `unique class code`. Using `SecureRandom` and `BigInteger` I created special unique class codes for **ONLY** "Teachers". Below is a code snippet of the following code - 

In [ ]:
@PostMapping( "/post")
public ResponseEntity<Object> postHuman(@RequestParam("email") String email,
                                         @RequestParam("password") String password,
                                         @RequestParam("name") String name,
                                         @RequestParam("dob") String dobString,
                                         @RequestParam("role") String role) {
    Date dob;
    try {
        dob = new SimpleDateFormat("MM-dd-yyyy").parse(dobString);
    } catch (Exception e) {
        return new ResponseEntity<>(dobString +" error; try MM-dd-yyyy", HttpStatus.BAD_REQUEST);
    }
    password = passwordEncoder.encode(password);
    List<Human> humans = repository.findAll();
    for (Human hman : humans){
        usedClassCodes.add(hman.getClassCode());
    }
    String classCode = "";  
    if (role.equals("Teacher")){

        int CODE_LENGTH = 6; 
        SecureRandom random = new SecureRandom();
        BigInteger randomBigInt;
        do {
            randomBigInt = new BigInteger(50, random);
            classCode = randomBigInt.toString(32).toUpperCase().substring(0, CODE_LENGTH);
        } while (usedClassCodes.contains(classCode));
        usedClassCodes.add(classCode);
    }
    else{
        if (role.equals("Student")){
        classCode = null;}
        else{
            return new ResponseEntity<>("The role has to be either 'Student' or 'Teacher'", HttpStatus.BAD_REQUEST);
        }
    }
    
    // A Human object WITHOUT ID will create a new record with default roles as student
    Human human = new Human(email, password, name, dob, role);
    human.setClassCode(classCode);
    repository.save(human);
    return new ResponseEntity<>(email +" created successfully", HttpStatus.CREATED);
}

### HumanDetailsService
This was another file that I took from person but added some changes to. One of the major changes is the `loadUserByUsername()` function. This fucntion gets called eachtime when a request is made with cookies to find the corresponding username from the JWT. So for some error handling and creation of `Authorities` for specific roles i added some System prints and using `getRole` and `SimpleGrantedAuthority` I made the roles of the Human object into an authority that I could use later. Below is the main code segment that was changed - 

In [ ]:
@Override
public org.springframework.security.core.userdetails.UserDetails loadUserByUsername(String email) throws UsernameNotFoundException {
    Human human = humanJpaRepository.findByEmail(email); // setting variable user equal to the method finding the username in the database
    if(human==null) {
        throw new UsernameNotFoundException("User not found with username: " + email);
    }

    // assigning roles authorities
    Collection<SimpleGrantedAuthority> authorities = new ArrayList<>();
    String role = human.getRole();
    
    // error checking and logging
    System.out.println(role);
    authorities.add(new SimpleGrantedAuthority(role));

    // more logging for checks
    System.out.println("Authorities:");
    for (SimpleGrantedAuthority authority : authorities) {
        System.out.println(authority.getAuthority());
    }
    // train spring security to User and Authorities
    return new org.springframework.security.core.userdetails.User(human.getEmail(), human.getPassword(), authorities);
}

## JWT Implementation
Various changes in files was refrenced or copied from person. the divergence occurs in the role authorities and the changes were the greatest in but not limited to these files: 

### JWTTokenUtil
This file had major changes to implement roles. We needed to assign the cookie that gets created `authorities/roles` from what their roles were set to in the database. We add it to the `claims` that becomes part of the cookie. When the JWT token or **cookie is sent back** in subsequent api requests we can check them and find their roles and authorities.  

### Security Config
Using the `authorities` granted and set up we can allow requests to **specfic api endpoints** to be accessed only by people have the specfic `role`

In [ ]:
public String generateToken(UserDetails userDetails) {
    Map<String, Object> claims = new HashMap<>();
    Collection<? extends GrantedAuthority> authorities = userDetails.getAuthorities();
    ArrayList<String> roles = new ArrayList<>();
    
    for (GrantedAuthority authority : authorities) {
        roles.add(authority.getAuthority());
    }

    claims.put("roles", roles);
    return doGenerateToken(claims, userDetails.getUsername());
}

private String doGenerateToken(Map<String, Object> claims, String subject) {

    return Jwts.builder().setClaims(claims).setSubject(subject).setIssuedAt(new Date(System.currentTimeMillis()))
            .setExpiration(new Date(System.currentTimeMillis() + JWT_TOKEN_VALIDITY * 1000))
            .signWith(SignatureAlgorithm.HS512, secret).compact();
}

// list the requests/endpoints need to be authenticated
			.authorizeRequests()
				.antMatchers("/mvc/person/update/**", "/mvc/person/delete/**").permitAll()
				.antMatchers("/api/person/**").permitAll()
				.antMatchers("/api/human/delete/**").hasAuthority("Teacher")
				.and()

## First Test Result:

### JWT token generation
![Image](https://github.com/aidenhuynh/cj_backend/assets/108041389/b4bc225a-292f-4f7d-850b-23acc8634e79)

### Roles and Authorities
![Image](https://github.com/aidenhuynh/cj_backend/assets/108041389/7958180f-0490-4721-a7ae-b5e474d192a7)



![Image](https://github.com/aidenhuynh/cj_backend/assets/108041389/3f7188ba-9b95-4f6d-9b8f-ea41c39705d1)



![Image](https://github.com/aidenhuynh/cj_backend/assets/108041389/9f64496a-4c62-4b7c-8d7a-069c28f392f5)



![Image](https://github.com/aidenhuynh/cj_backend/assets/108041389/04519b32-7c9b-4c70-8a5a-7184939c8bb4)



## Commits:


![Image](https://github.com/aidenhuynh/cj_backend/assets/108041389/927f7d88-59bd-445d-87de-e230b3fb9995)



![Image](https://github.com/aidenhuynh/cj_backend/assets/108041389/3b0dfef3-006a-4be4-950a-4978d1ed6835)